In order to use dataframes and the SQL engine in Apache Spark (SparkSQL), we need a Spark Session.


In [3]:
import pyspark

In [4]:
from pyspark.sql import SparkSession

In [18]:
spark = SparkSession.builder.appName('Practice').getOrCreate()

In [19]:
spark

## Spark Dataframes

A DataFrame is two-dimensional. Columns can be of different data types. DataFrames accept many data inputs including series and other DataFrames. You can pass indexes (row labels) and columns (column labels). Indexes can be numbers, dates, or strings/tuples.

### Read dataset as Spark dataframe from file

In [7]:
# Read Dataset as Spark dataframe from json file
df = spark.read.json('people.json')

In [8]:
df.show()
df.head(2)
df.printSchema()
df.dtypes

+---+-------+
|age|   name|
+---+-------+
| 25|Michael|
| 24|   Andy|
| 19| Justin|
| 26| George|
| 30|   Jeff|
+---+-------+

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



[('age', 'bigint'), ('name', 'string')]

In [9]:
# Read Dataset as Spark dataframe from csv file
df = spark.read.option('header','true').csv('people.csv', inferSchema=True)

In [10]:
df.show()
df.head(2)
df.printSchema()
df.dtypes

+-------+---+
|   name|age|
+-------+---+
|Michael| 25|
|   Andy| 24|
| Justin| 19|
| George| 26|
|   Jeff| 30|
+-------+---+

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)



[('name', 'string'), ('age', 'int')]

In [11]:
df.select("name").show()

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
| George|
|   Jeff|
+-------+



In [13]:
df.select(df["name"]).show()

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
| George|
|   Jeff|
+-------+



### Add and drop a column

In [14]:
# add a column

from pyspark.sql.functions import length

df = df.withColumn('name_length', length(df['name']))
df.show()

+-------+---+-----------+
|   name|age|name_length|
+-------+---+-----------+
|Michael| 25|          7|
|   Andy| 24|          4|
| Justin| 19|          6|
| George| 26|          6|
|   Jeff| 30|          4|
+-------+---+-----------+



In [15]:
# drop a column

df = df.drop('name_length')
df.show()

+-------+---+
|   name|age|
+-------+---+
|Michael| 25|
|   Andy| 24|
| Justin| 19|
| George| 26|
|   Jeff| 30|
+-------+---+



In [16]:
spark.stop()

### Missing Values

In [25]:
df = spark.read.csv('class_grades.csv', header=True, inferSchema=True)

In [26]:
df.show(10)

+------+----------+--------+-------+--------+-----+
|Prefix|Assignment|Tutorial|Midterm|TakeHome|Final|
+------+----------+--------+-------+--------+-----+
|     5|     57.14|   34.09|  64.38|   51.48| 52.5|
|     8|     95.05|  105.49|   67.5|   99.07|68.33|
|     8|      83.7|   83.17|   30.0|   63.15|48.89|
|     7|     81.22|   96.06|  49.38|  105.93|80.56|
|     8|     91.32|   93.64|   95.0|  107.41|73.89|
|     7|      95.0|   92.58|  93.12|   97.78|68.06|
|     8|     95.05|  102.99|  56.25|   99.07| 50.0|
|     7|     72.85|   86.85|   60.0|    NULL|56.11|
|     8|     84.26|    93.1|   47.5|   18.52|50.83|
|     7|      90.1|   97.55|  51.25|   88.89|63.61|
+------+----------+--------+-------+--------+-----+
only showing top 10 rows



In [27]:
df = df.na.drop()

In [28]:
df.show(10)

+------+----------+--------+-------+--------+-----+
|Prefix|Assignment|Tutorial|Midterm|TakeHome|Final|
+------+----------+--------+-------+--------+-----+
|     5|     57.14|   34.09|  64.38|   51.48| 52.5|
|     8|     95.05|  105.49|   67.5|   99.07|68.33|
|     8|      83.7|   83.17|   30.0|   63.15|48.89|
|     7|     81.22|   96.06|  49.38|  105.93|80.56|
|     8|     91.32|   93.64|   95.0|  107.41|73.89|
|     7|      95.0|   92.58|  93.12|   97.78|68.06|
|     8|     95.05|  102.99|  56.25|   99.07| 50.0|
|     8|     84.26|    93.1|   47.5|   18.52|50.83|
|     7|      90.1|   97.55|  51.25|   88.89|63.61|
|     7|     80.44|    90.2|   75.0|   91.48|39.72|
+------+----------+--------+-------+--------+-----+
only showing top 10 rows



## Filtering

In [31]:
df.filter(df["Final"] > 60).show(10)

+------+----------+--------+-------+--------+-----+
|Prefix|Assignment|Tutorial|Midterm|TakeHome|Final|
+------+----------+--------+-------+--------+-----+
|     8|     95.05|  105.49|   67.5|   99.07|68.33|
|     7|     81.22|   96.06|  49.38|  105.93|80.56|
|     8|     91.32|   93.64|   95.0|  107.41|73.89|
|     7|      95.0|   92.58|  93.12|   97.78|68.06|
|     7|      90.1|   97.55|  51.25|   88.89|63.61|
|     6|     86.26|    80.6|  74.38|   87.59| 77.5|
|     8|     97.16|  103.71|   72.5|   93.52|63.33|
|     7|     91.28|   83.53|  81.25|   99.81|92.22|
|     7|     93.83|   95.43|  88.12|   80.93| 90.0|
|     8|     55.14|   81.85|   75.0|   56.11| 62.5|
+------+----------+--------+-------+--------+-----+
only showing top 10 rows



## Group By and Aggregation

In [32]:
df.groupBy("Prefix").avg('Final').show()

+------+-----------------+
|Prefix|       avg(Final)|
+------+-----------------+
|     6|60.63428571428572|
|     5|           51.665|
|     4|57.68666666666667|
|     8|67.06099999999999|
|     7|74.47848484848484|
+------+-----------------+

